In [13]:
# code for autoreload of modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os


In [15]:
DIR = r'C:\Users\u0148775\PycharmProjects\CreditScoring'
os.chdir(DIR)


In [16]:
from src.utils import load_config

# read dataconfig from CONFIG_DATA.yaml file:

dataconfig = load_config('config\CONFIG_DATA.yaml')
experimentconfig = load_config('config\CONFIG_EXPERIMENT.yaml')
methodconfig = load_config('config\CONFIG_METHOD.yaml')
evaluationconfig = load_config('config\CONFIG_EVALUATION.yaml')



In [17]:
from src.classes.experiment import Experiment

# Initialize experiment object
experiment = Experiment(dataconfig=dataconfig,
                        experimentconfig=experimentconfig,
                        methodconfig=methodconfig,
                        evaluationconfig=evaluationconfig)



Experiment created at:  2025-02-18-17-42
Task: pd
Dataset: 03_vehicle_loan
CV splits:  2


In [18]:
experiment.run()

03_vehicle_loan loaded
03_vehicle_loan preprocessed
x shape:  (233154, 33)
y shape:  (233154,)


Cross-validation loop::   0%|          | 0/2 [00:00<?, ?it/s]

Omitting rows with missing values: 7661 rows left out
*Best hyperparameters (dt)* {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}
*Best hyperparameters (lgbm)* {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 50, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1}


Cross-validation loop::  50%|█████     | 1/2 [00:07<00:07,  7.16s/it]

Omitting rows with missing values: 7661 rows left out
*Best hyperparameters (dt)* {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}
*Best hyperparameters (lgbm)* {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1}


Cross-validation loop:: 100%|██████████| 2/2 [00:14<00:00,  7.31s/it]


In [20]:
experiment.results

{'dt': {0: {'accuracy': 0.73383,
   'brier': 0.23627,
   'f1': 0.1819,
   'precision': 0.27361,
   'recall': 0.27361,
   'aucroc': 0.51794,
   'aucpr': 0.22488},
  1: {'accuracy': 0.73635,
   'brier': 0.22867,
   'f1': 0.16975,
   'precision': 0.26833,
   'recall': 0.26833,
   'aucroc': 0.51513,
   'aucpr': 0.22347}},
 'lgbm': {0: {'accuracy': 0.78292,
   'brier': 0.16196,
   'f1': 0.01425,
   'precision': 0.51906,
   'recall': 0.51906,
   'aucroc': 0.50268,
   'aucpr': 0.21938},
  1: {'accuracy': 0.78254,
   'brier': 0.16225,
   'f1': 0.02312,
   'precision': 0.4685,
   'recall': 0.4685,
   'aucroc': 0.50406,
   'aucpr': 0.22009}}}